Setup

In [2]:
# imports
from pathlib import Path
import os

# Install dependencies and validate runtime (GPU/CPU)
import sys
import subprocess
import importlib

# Runtime checks
import torch
from ultralytics import YOLO

import roboflow

In [3]:
# Project metadata
PROJECT_NAME = "yolov8n-finetune-134"
CLASSES = ["w", "y", "r", "o", "b", "g"]

# Model/training intent
MODEL_NAME = "yolov8n.pt"  # nano model for edge constraints
IMAGE_SIZE = 640

print("Project:", PROJECT_NAME)
print("Model:", MODEL_NAME)
print("Classes:", CLASSES)

Project: yolov8n-finetune-134
Model: yolov8n.pt
Classes: ['w', 'y', 'r', 'o', 'b', 'g']


In [4]:
print("Python executable:", sys.executable)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("CUDA device name:", torch.cuda.get_device_name(0))
    DEVICE = 0
else:
    print("No CUDA GPU detected. Training will run on CPU (slower).")
    DEVICE = "cpu"

# sanity check
_ = YOLO(MODEL_NAME)
print(f"Model '{MODEL_NAME}' loaded successfully.")
print("Selected training device:", DEVICE)

Python executable: c:\Python313\python.exe
PyTorch version: 2.10.0+cu128
CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA GeForce RTX 4060 Laptop GPU
Model 'yolov8n.pt' loaded successfully.
Selected training device: 0


Finetune

In [ ]:
# Use local dataset you downloaded manually
DATA_YAML_PATH = str(Path.cwd() / "data" / "data.yaml")
assert Path(DATA_YAML_PATH).exists(), f"Missing dataset config: {DATA_YAML_PATH}"

# hyperparameters
EPOCHS = 80
BATCH_SIZE = 16
PATIENCE = 20
LR0 = 0.005

model = YOLO(MODEL_NAME)

results = model.train(
    data=DATA_YAML_PATH,
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    workers=8,
    pretrained=True,
    optimizer="AdamW",
    lr0=LR0,
    patience=PATIENCE,
    cos_lr=True,
    weight_decay=0.0005,
    project="runs",
    name="rubiks_yolov8n",
    exist_ok=True,
    amp=True,
    verbose=True,
)

print("Training complete.")
print("Best weights:", Path("runs") / "rubiks_yolov8n" / "weights" / "best.pt")
print("Last weights:", Path("runs") / "rubiks_yolov8n" / "weights" / "last.pt")

Ultralytics 8.4.14  Python-3.13.2 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=c:\Users\adity\Desktop\134YOLO\data\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=80, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=rubiks_yolov8n, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW,

In [5]:
BEST_WEIGHTS = Path("runs") / "detect" / "runs" / "rubiks_yolov8n" / "weights" / "best.pt"
assert BEST_WEIGHTS.exists(), f"best.pt not found at: {BEST_WEIGHTS}"

model = YOLO(str(BEST_WEIGHTS))

test_metrics = model.val(
    data=str(Path("data") / "data.yaml"),
    split="test",        # held-out test set
    imgsz=IMAGE_SIZE,
    batch=16,
    device=DEVICE,
    conf=0.001,
    iou=0.6,
    plots=True,
    verbose=True,
)

print("\n=== Test Metrics (best.pt) ===")
print(f"mAP50:    {test_metrics.box.map50:.4f}")
print(f"mAP50-95: {test_metrics.box.map:.4f}")
print(f"Precision:{test_metrics.box.mp:.4f}")
print(f"Recall:   {test_metrics.box.mr:.4f}")

print("\nPer-class AP50:")
for class_name, ap50 in zip(test_metrics.names.values(), test_metrics.box.ap50):
    print(f"  {class_name}: {ap50:.4f}")

print("\nSaved eval artifacts under the run directory shown above.")

Ultralytics 8.4.14  Python-3.13.2 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 73 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 1.90.4 MB/s, size: 35.2 KB)
val: Scanning C:\Users\adity\Desktop\134YOLO\data\test\labels.cache... 150 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 150/150 14.0Mit/s 0.0s
WARNING Box and segment counts should be equal, but got len(segments) = 9, len(boxes) = 2232. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 3.4it/s 3.0s.2ss
                   all        150       2232      0.987      0.985      0.989      0.796
                     b        136        354      0.994       0.99       0.99      0.792
           

In [8]:
# pick photos to inference by placing in /data/inference

BEST_WEIGHTS = Path("runs") / "detect" / "runs" / "rubiks_yolov8n" / "weights" / "best.pt"
INFER_DIR = Path("data") / "inference"

assert BEST_WEIGHTS.exists(), f"best.pt not found: {BEST_WEIGHTS}"
assert INFER_DIR.exists(), f"Inference folder not found: {INFER_DIR}"

# Collect supported image files
exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
image_files = sorted([p for p in INFER_DIR.iterdir() if p.suffix.lower() in exts])
assert len(image_files) > 0, f"No images found in {INFER_DIR}"

print(f"Found {len(image_files)} image(s) in {INFER_DIR}")

model = YOLO(str(BEST_WEIGHTS))

results = model.predict(
    source=[str(p) for p in image_files],
    imgsz=IMAGE_SIZE,
    conf=0.25,
    iou=0.6,
    device=DEVICE,
    save=True,
    save_txt=True,
    save_conf=True,
    project="runs",
    name="rubiks_inference",
    exist_ok=True,
    verbose=False,
)

print("\nInference complete.")
print("Annotated outputs:", Path("runs") / "detect" / "runs" / "rubiks_inference")

# Print concise per-image predictions
for r in results:
    img_name = Path(r.path).name
    print(f"\n{img_name}")
    if r.boxes is None or len(r.boxes) == 0:
        print("  No detections")
        continue

    cls_ids = r.boxes.cls.tolist()
    confs = r.boxes.conf.tolist()

    for cid, c in zip(cls_ids, confs):
        label = r.names[int(cid)]
        print(f"  {label}: {c:.3f}")

Found 3 image(s) in data\inference
Results saved to C:\Users\adity\Desktop\134YOLO\runs\detect\runs\rubiks_inference
3 labels saved to C:\Users\adity\Desktop\134YOLO\runs\detect\runs\rubiks_inference\labels

Inference complete.
Annotated outputs: runs\detect\runs\rubiks_inference

51TDx5JhLYL._AC_UF894,1000_QL80_.jpg
  b: 0.851
  w: 0.847
  w: 0.837
  b: 0.834
  w: 0.832
  b: 0.828
  b: 0.826
  w: 0.811
  b: 0.804
  w: 0.802
  w: 0.799
  w: 0.792
  w: 0.785
  w: 0.762
  b: 0.757
  b: 0.742
  w: 0.728
  w: 0.442
  w: 0.302

51xT0BvVxCL.jpg
  r: 0.817
  w: 0.816
  r: 0.814
  r: 0.812
  w: 0.811
  r: 0.810
  r: 0.808
  w: 0.801
  r: 0.796
  w: 0.781
  w: 0.770
  w: 0.769
  r: 0.758
  r: 0.758
  w: 0.738
  r: 0.738
  w: 0.694
  r: 0.685
  r: 0.681
  r: 0.652
  w: 0.605
  r: 0.533
  r: 0.506
  r: 0.486
  r: 0.485
  g: 0.437
  g: 0.311
  b: 0.278
  g: 0.258

scrambleface.webp
  y: 0.901
  b: 0.877
  y: 0.869
  b: 0.855
  g: 0.848
  w: 0.843
  b: 0.838
  g: 0.833
  g: 0.832
